In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np

In [ ]:
image = tf.keras.utils.image_dataset_from_directory(
    "./drive/MyDrive/Image/", labels=None, label_mode=None,
    class_names=None, color_mode="grayscale", batch_size=1, image_size=(256,
    256), shuffle=True, seed=None, validation_split=None, subset=None,
    interpolation='bilinear', follow_links=False,
    crop_to_aspect_ratio=False)


The convolutional autoencoder

In [ ]:
from tensorflow import keras


In [ ]:
def c_auto_encoder():
  c_encoder = keras.models.Sequential([
    keras.layers.Input(shape=[256,256,1]),
    keras.layers.Conv2D(16,kernel_size=2,padding="same",activation="selu"),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(8,kernel_size=2,padding="same",activation="selu"),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(8,kernel_size=2,padding="same",activation="selu"),
    keras.layers.MaxPool2D(pool_size=2)                               
  ])

  c_decoder = keras.models.Sequential([
    #keras.layers.UpSampling2D(size=(2,2)),
    keras.layers.Conv2D(8,kernel_size=2,padding="same",activation="selu",input_shape=[32,32,8]),
    keras.layers.UpSampling2D(size = (2,2)),
    keras.layers.Conv2D(8,kernel_size=2,strides=(2,2),padding="same",activation="selu"),
    keras.layers.UpSampling2D(size = (2,2)),
    keras.layers.Conv2D(16,kernel_size=2,strides=(2,2),padding="same",activation="selu"),
    keras.layers.UpSampling2D(size = (2,2)),
    keras.layers.UpSampling2D(size = (2,2)),
    keras.layers.UpSampling2D(size = (2,2)),
    keras.layers.UpSampling2D(size = (2,2)),
    keras.layers.Conv2D(1,kernel_size=2,strides=2,padding="same",activation="sigmoid"),
    
    #keras.layers.Reshape([3840,2160,1],input_shape=[3840,2160,1])
  ])

  c_auto_encoder = keras.models.Sequential([c_encoder,c_decoder])
  c_auto_encoder.compile(loss=keras.losses.BinaryCrossentropy(),optimizer = keras.optimizers.Adam(learning_rate=0.001)) 
  return c_auto_encoder

In [ ]:
ae = c_auto_encoder()

In [ ]:
ae.fit(image,epochs=50)

In [ ]:
c_decoder = keras.models.Sequential([
    #keras.layers.UpSampling2D(size=(2,2)),
    keras.layers.Conv2D(8,kernel_size=2,padding="same",activation="selu",input_shape=[32,32,8]),
    keras.layers.UpSampling2D(size = (2,2)),
    keras.layers.Conv2D(8,kernel_size=2,strides=(2,2),padding="same",activation="selu"),
    keras.layers.UpSampling2D(size = (2,2)),
    keras.layers.Conv2D(16,kernel_size=2,strides=(2,2),padding="same",activation="selu"),
    keras.layers.UpSampling2D(size = (2,2)),
    keras.layers.UpSampling2D(size = (2,2)),
    keras.layers.UpSampling2D(size = (2,2)),
    keras.layers.UpSampling2D(size = (2,2)),
    keras.layers.Conv2D(1,kernel_size=2,strides=2,padding="same",activation="sigmoid"),
    
    #keras.layers.Reshape([3840,2160,1],input_shape=[3840,21600,16])
  ])
c_decoder.summary()

In [ ]:
c_encoder = keras.models.Sequential([
    keras.layers.Input(shape=[256,256,1]),
    keras.layers.Conv2D(16,kernel_size=2,padding="same",activation="selu"),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(8,kernel_size=2,padding="same",activation="selu"),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(8,kernel_size=2,padding="same",activation="selu"),
    keras.layers.MaxPool2D(pool_size=2)                               
  ])
c_encoder.summary()

In [ ]:
c_auto_encoder = keras.models.Sequential([c_encoder,c_decoder])
c_auto_encoder.build(input_shape=[32,3840,2160,1])
c_auto_encoder.summary()

A simple GAN

In [ ]:
import numpy as np
from tensorflow import keras
generator = keras.models.Sequential([
    keras.layers.Dense(100,activation="selu",input_shape=[30]),
    keras.layers.Dense(150,activation="selu"),
    keras.layers.Dense(256*256,activation="sigmoid"),
    keras.layers.Reshape([256,256,1])
    ])
generator.summary()

In [ ]:
discriminator = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[256,256,1]),
    keras.layers.Dense(100,activation="selu"),
    keras.layers.Dense(150,activation="selu"),
    keras.layers.Dense(1,activation="sigmoid")
])
discriminator.summary()

In [ ]:
gan = keras.models.Sequential(generator,discriminator)
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [ ]:
def train(gan,image,batch_size,generator,discriminator,n_epochs = 50):
  #generator,discriminator = gan.layers
  for epoch in range(n_epochs):
    for batch in image:
      noise = np.random.normal(size=[batch_size,30])
      generate =generator(noise)
      fake_and_real = np.concatenate([generate,batch],axis = 0)
      y1 = np.array([[0.]]*batch_size+[[1.]]*batch_size)

      discriminator.trainable = True
      discriminator.train_on_batch(fake_and_real, y1)
      noise = np.random.normal(size=[batch_size,30])
      y2 = np.array([[1.]]*batch_size)
      discriminator.trainable = False
      gan.train_on_batch(noise,y2)

train(gan,image,1,generator,discriminator)